In [19]:
import numpy as np
from scipy.optimize import minimize_scalar

## Task 1: Crude Monte Carlo

In [20]:
exact_value = np.exp(1) - 1 # exact value of the integral from 0 to 1 of e^x

In [21]:
n = 100
samples = np.random.uniform(0, 1, n)
f_values = np.exp(samples)
estimate = np.mean(f_values)
std_dev = np.std(f_values, ddof=1)
conf_interval = (estimate - 1.96 * std_dev / np.sqrt(n), estimate + 1.96 * std_dev / np.sqrt(n))

In [22]:
print(f"Estimate of int = E[e^X]: {estimate}")
print(f"95% confidence interval: {conf_interval}")
print(f"Confidence interval width: {conf_interval[1] - conf_interval[0]}")
print(f"Error from exact value: {estimate - exact_value}")

Estimate of int = E[e^X]: 1.7258371186819246
95% confidence interval: (1.6353817241966986, 1.8162925131671506)
Confidence interval width: 0.18091078897045199
Error from exact value: 0.007555290222879485


## Task 2: Antithetic variables

In [23]:
m = n // 2 # 100 function evaluations, so we generate 50 random values
u = np.random.uniform(0, 1, m)
f_values = (np.exp(u) + np.exp(1 - u)) / 2
estimate = np.mean(f_values)
std_dev = np.std(f_values, ddof=1)
conf_interval = (estimate - 1.96 * std_dev / np.sqrt(m), estimate + 1.96 * std_dev / np.sqrt(m))

In [24]:
print(f"Estimate of int = E[e^X]: {estimate}")
print(f"95% confidence interval: {conf_interval}")
print(f"Confidence interval width: {conf_interval[1] - conf_interval[0]}")
print(f"Error from exact value: {estimate - exact_value}")

Estimate of int = E[e^X]: 1.727874528575985
95% confidence interval: (1.7069281180731772, 1.7488209390787925)
Confidence interval width: 0.04189282100561531
Error from exact value: 0.0095927001169398


## Task 3: Control variable

In [25]:
U = np.random.uniform(0, 1, n)
X = np.exp(U)
Y = U
mu_Y = 0.5 # known mean of Y

# estimate optimal c
cov_XY = np.cov(X, Y, ddof=1)[0, 1]
var_Y = np.var(Y, ddof=1)
c = -cov_XY / var_Y 

# construct Z_i and estimate its mean
Z = X + c * (Y - mu_Y)
estimate = np.mean(Z)
std_dev = np.std(Z, ddof=1)
conf_interval = (estimate - 1.96 * std_dev / np.sqrt(n), estimate + 1.96 * std_dev / np.sqrt(n))

In [26]:
print(f"Estimate of int = E[e^X]: {estimate}")
print(f"95% confidence interval: {conf_interval}")
print(f"Confidence interval width: {conf_interval[1] - conf_interval[0]}")
print(f"Error from exact value: {estimate - exact_value}")

Estimate of int = E[e^X]: 1.7199960030530008
95% confidence interval: (1.7072200938765887, 1.732771912229413)
Confidence interval width: 0.025551818352824274
Error from exact value: 0.0017141745939557307


## Task 4: Stratified sampling

In [27]:
# number of strata (and samples)
k = 100
U = np.random.uniform(0, 1, k)

x_strata = (np.arange(k) + U) / k  # (j - 1 + U_j) / k
f_vals = np.exp(x_strata)

estimate = np.mean(f_vals)
std_dev = np.std(f_vals, ddof=1)
conf_interval = (estimate - 1.96 * std_dev / np.sqrt(k), estimate + 1.96 * std_dev / np.sqrt(k))

In [28]:
print(f"Estimate of int = E[e^X]: {estimate}")
print(f"95% confidence interval: {conf_interval}")
print(f"Confidence interval width: {conf_interval[1] - conf_interval[0]}")
print(f"Error from exact value: {estimate - exact_value}")

Estimate of int = E[e^X]: 1.7169735989631407
95% confidence interval: (1.6203383034404866, 1.8136088944857949)
Confidence interval width: 0.1932705910453083
Error from exact value: -0.0013082294959043672


## Task 5: Control variates for previous exercise

See end of notebook 4

## Task 6: CRN in Exercise 4

See end of notebook 4

## Task 7: Importance sampling vs crude Monte Carlo

In [31]:
# from exercise 3:
def simulate_normal_box_muller(n):
    U1 = np.random.uniform(size=n//2)
    U2 = np.random.uniform(size=n//2)

    R = np.sqrt(-2 * np.log(U1))
    theta = 2 * np.pi * U2

    Z1 = R * np.cos(theta)
    Z2 = R * np.sin(theta)

    Z = np.concatenate([Z1, Z2])
    return Z 

# pdf normal
def normal_pdf(x, mu=0, sigma=1):
    coeff = 1.0 / (np.sqrt(2 * np.pi) * sigma)
    exponent = -((x - mu) ** 2) / (2 * sigma ** 2)
    return coeff * np.exp(exponent)

In [32]:
# define parameters
a_values = [2, 4]  # thresholds
sample_sizes = [100, 1000, 10000]
sigma2 = 1  # for importance sampling

results = []

for a in a_values:
    for n in sample_sizes:

        # crude Monte Carlo:
        Z = simulate_normal_box_muller(n)
        crude_probs = Z > a
        crude_estimate = np.mean(crude_probs)
        crude_std_error = np.std(crude_probs, ddof=1) / np.sqrt(n)

        # importance sampling:
        mu = a 
        sigma = np.sqrt(sigma2)

        # simulate standard normals using Box-Muller
        Z = simulate_normal_box_muller(n)
        # shift and scale to sample from g = N(mu, sigma^2)
        Y = mu + sigma * Z

        # compute weights f(Y) / g(Y)
        f_Y = normal_pdf(Y, mu=0, sigma=1) # standard normal
        g_Y = normal_pdf(Y, mu=mu, sigma=sigma) # importance distribution
        weights = f_Y / g_Y

        indicators = (Y > a)
        imp_estimate = np.mean(indicators * weights)
        imp_std_error = np.std(indicators * weights, ddof=1) / np.sqrt(n)      

        results.append({
            'a': a,
            'n': n,
            'crude_estimate': crude_estimate,
            'crude_std_error': crude_std_error,
            'importance_estimate': imp_estimate,
            'importance_std_error': imp_std_error
        })

In [33]:
for r in results:
    print(f"a = {r['a']}, n = {r['n']}")
    print(f"  Crude MC estimate: {r['crude_estimate']:.6f} ± {1.96*r['crude_std_error']:.6f}")
    print(f"  IS estimate      : {r['importance_estimate']:.6f} ± {1.96*r['importance_std_error']:.6f}")
    print()

a = 2, n = 100
  Crude MC estimate: 0.010000 ± 0.019600
  IS estimate      : 0.022793 ± 0.007447

a = 2, n = 1000
  Crude MC estimate: 0.017000 ± 0.008016
  IS estimate      : 0.023493 ± 0.002201

a = 2, n = 10000
  Crude MC estimate: 0.019500 ± 0.002710
  IS estimate      : 0.021763 ± 0.000666

a = 4, n = 100
  Crude MC estimate: 0.000000 ± 0.000000
  IS estimate      : 0.000038 ± 0.000015

a = 4, n = 1000
  Crude MC estimate: 0.000000 ± 0.000000
  IS estimate      : 0.000029 ± 0.000004

a = 4, n = 10000
  Crude MC estimate: 0.000000 ± 0.000000
  IS estimate      : 0.000031 ± 0.000001



- Crude MC works okay for moderate tail probabilities but fails for very rare events. (In the case where a = 4). Confidence interval captures true value for Crude MC for a = 2. 

- Importance sampling significantly reduces variance for rare-event probabilities and provides meaningful estimates even with modest samples.


## Task 8: Importance sampling exponential

In [34]:
# from Exercise 3
def simulate_exponential(n, lam):
    U = np.random.uniform(size=n)
    X = -np.log(U) / lam
    return X

The estimator is
$$
\theta = \frac{1}{n} \sum_{i=1}^n h(Y_i) \frac{f(Y_i)}{g(Y_i)}, \quad Y_i \sim g
$$

- $h(x) = \mathbf{1}_{x \leq 1}$ because the integral is over [0,1].
- $f(x) = e^x$
-  $g(x) = \lambda e^{-\lambda x}$

We sample $Y_i$ from the exponential distribution $g$ and weight samples inside the integration domain \[0,1]:

$$
\hat{\theta} = \frac{1}{n} \sum_{i=1}^n \mathbf{1}_{Y_i \leq 1} \cdot e^{Y_i} \frac{1}{\lambda e^{-\lambda Y_i}} = \frac{1}{n} \sum_{i=1}^n \mathbf{1}_{Y_i \leq 1} \frac{e^{Y_i(1+\lambda)}}{\lambda}
$$



To find the optimal $\lambda$ for importance sampling with $g(x) = \lambda e^{-\lambda x}$ when estimating $\int_0^1 e^x dx$, we analyze the variance of $W = \frac{h(x)f(x)}{g(x)}$

$$
W = \mathbf{1}_{x \leq 1} \frac{e^{x(1+\lambda)}}{\lambda}
$$

The variance is

$$
\text{Var}(W) = E_g[W^2] - (E_g[W])^2 
$$

We have
\begin{align*}
    E_g[W^2] &= \int_0^1 \left( \frac{e^{x(1+\lambda)}}{\lambda} \right)^2 \cdot \lambda e^{-\lambda x} dx = \frac{1}{\lambda^2} \int_0^1 e^{2x(1+\lambda)} \cdot \lambda e^{-\lambda x} dx \\
    & = \frac{\lambda}{\lambda^2} \int_0^1 e^{(2+\lambda) x} dx = \frac{1}{\lambda} \int_0^1 e^{(2+\lambda) x} dx  = \frac{e^{2+\lambda} - 1}{2 + \lambda}
\end{align*}

Therefore, 

$$
\text{Var}(W) = E_g[W^2] - (E_g[W])^2 = \frac{1}{\lambda} \cdot \frac{e^{2+\lambda} - 1}{2 + \lambda} - (e - 1)^2
$$


We can numerically find the $\lambda$ that minimizes this variance. However, as expected from theory, using the exponential distribution $g$ here does not significantly reduce variance compared to crude Monte Carlo, because many samples fall outside the interval $[0,1]$ and contribute zero to the estimator.


In [35]:
def variance_W(lam):
    if lam <= 0:
        return np.inf
    EgW2 = (1 / lam) * (np.exp(2 + lam) - 1) / (2 + lam)
    EgW = np.exp(1) - 1
    var = EgW2 - EgW**2
    return var

# minimize variance over lambda > 0
res = minimize_scalar(variance_W, bounds=(0.01, 10), method='bounded')
optimal_lambda = res.x

print(f"Optimal lambda: {optimal_lambda:.4f}")

Optimal lambda: 1.3548


In [36]:
def importance_sampling_exp(lam, n=10000):
    X = simulate_exponential(n, lam)
    indicators = (X <= 1)
    weights = indicators * (np.exp(X * (1 + lam)) / lam)
    estimate = np.mean(weights)
    std_error = np.std(weights, ddof=1) / np.sqrt(n)
    return estimate, std_error

# run for some lambdas
lam_values = [0.1, 0.5, 1.0, 1.3548, 2.0, 5.0]
n = 10000

print("Importance sampling estimates for various lambda:")
for lam in lam_values:
    est_is, se_is = importance_sampling_exp(lam, n)
    print(f"lambda={lam:.4f}: Estimate = {est_is:.6f} ± {se_is:.6f}")


Importance sampling estimates for various lambda:
lambda=0.1000: Estimate = 1.776642 ± 0.057055
lambda=0.5000: Estimate = 1.728338 ± 0.024458
lambda=1.0000: Estimate = 1.729109 ± 0.018409
lambda=1.3548: Estimate = 1.750560 ± 0.018141
lambda=2.0000: Estimate = 1.727724 ± 0.019663
lambda=5.0000: Estimate = 1.762760 ± 0.053452


Optimal value for lambda seems correct from simulation/ test of different lambda values. 

## Task 9: Importance sampling Pareto

Given a Pareto-distributed random variable:

$$
X \sim \text{Pareto}(k, \beta),
$$

with PDF:

$$
f(x) = \frac{k}{\beta} \left(\frac{x}{\beta}\right)^{-k-1}, \quad x \ge \beta,
$$

and CDF:

$$
F(x) = 1 - \left(\frac{\beta}{x}\right)^k.
$$

We want to estimate the mean $\mathbb{E}[X]$ using importance sampling, where the sampling distribution is the first moment distribution of $X$.

For nonnegative random variables, the first moment distribution is defined as (from lecture notes):

$$
G_1(x) = \frac{1}{\mathbb{E}[X]} \int_{\beta}^x t f(t) \, dt,
$$

where $\mathbb{E}[X]$ is the first moment of $X$, and the integral gives the contribution to the first moment from values $\leq x$.

We have:

$$
\mathbb{E}[X] = \int_{\beta}^{\infty} x f(x) \, dx = \frac{k \beta}{k - 1}, \quad \text{for } k > 1.
$$

Now compute $G_1(x)$:

$$
G_1(x) = \frac{1}{\mathbb{E}[X]} \int_{\beta}^{x} t f(t) \, dt = \frac{1}{\frac{k \beta}{k - 1}} \int_{\beta}^{x} t \cdot \frac{k}{\beta} \left(\frac{t}{\beta}\right)^{-k-1} dt.
$$

Rewrite the integral:

$$
= \frac{k - 1}{k \beta} \cdot k \beta^k \int_{\beta}^{x} t^{-k} dt = (k - 1) \beta^{k - 1} \int_{\beta}^{x} t^{-k} dt.
$$

Integrating:

$$
\int_{\beta}^{x} t^{-k} dt = \left[ \frac{t^{-k + 1}}{-k + 1} \right]_{\beta}^{x} = \frac{1}{k - 1} \left(\beta^{-k + 1} - x^{-k + 1}\right),
$$

so:

$$
G_1(x) = (k - 1) \beta^{k - 1} \cdot \frac{1}{k - 1} \left(\beta^{-k + 1} - x^{-k + 1}\right) = 1 - \left(\frac{\beta}{x}\right)^{k - 1}.
$$

Thus, the CDF of the first moment distribution is:

$$
G_1(x) = 1 - \left(\frac{\beta}{x}\right)^{k - 1}, \quad x \ge \beta,
$$

which is again a Pareto distribution with shape parameter $k - 1$. So:

$$
X_{\text{IS}} \sim \text{Pareto}(k - 1, \beta).
$$

We now estimate the mean using importance sampling:

$$
\mathbb{E}[X] = \int_{\beta}^{\infty} x f(x) \, dx = \int_{\beta}^{\infty} x \cdot \frac{f(x)}{g(x)} \cdot g(x) \, dx = \mathbb{E}_g\left[x \cdot \frac{f(x)}{g(x)}\right],
$$

where

$$
g(x) = \frac{d}{dx} G_1(x) = \frac{(k - 1) \beta^{k - 1}}{x^{k}}.
$$

Now compute the weight:

$$
\frac{f(x)}{g(x)} = \frac{\frac{k}{\beta} \left(\frac{x}{\beta}\right)^{-k-1}}{\frac{(k - 1) \beta^{k - 1}}{x^{k}}} = \frac{\frac{k}{\beta} \cdot \beta^{k+1} x^{-k-1}}{\frac{(k-1) \beta^{k-1}}{x^{k}}} = \frac{k}{k-1} \cdot \frac{\beta}{x}.
$$

Therefore, the IS estimator is:

$$
\hat{\mu}_{\text{IS}} = \frac{1}{n} \sum_{i=1}^n X_i \cdot \frac{f(X_i)}{g(X_i)} = \frac{1}{n} \sum_{i=1}^n X_i \cdot \frac{k}{k - 1} \cdot \frac{\beta}{X_i} = \frac{k}{k - 1} \cdot \beta.
$$

The estimator simplifies to a constant, meaning that this specific importance sampling strategy gives us the exact value of the mean without any sampling error. This makes sense, since the proposal distribution is chosen to be proportional to the integrand, it's effectively the optimal choice for this problem. 

A better choice of $g(x)$ in Question 8 would be the first moment distribution on $[0, 1]$, so we give more weight to larger values of $x$, where $e^x$ is large, and align the sampling distribution with the shape of the integrand.

Let $X \sim \text{Uniform}(0, 1)$, with density $f(x) = 1$ on $[0,1] $. The first moment distribution has CDF:

$$
G_1(x) = \frac{1}{\mathbb{E}[X]} \int_0^x t f(t) \, dt = \frac{1}{\mathbb{E}[X]} \int_0^x t \, dt
$$

We compute:

$$
\mathbb{E}[X] = \int_0^1 x \cdot 1 \, dx = \frac{1}{2}
$$

So:

$$
G_1(x) = \frac{1}{1/2} \int_0^x t \, dt = 2 \cdot \left[ \frac{t^2}{2} \right]_0^x = x^2
$$

Thus, the CDF of the first moment distribution is:

$$
G_1(x) = x^2, \quad x \in [0, 1]
$$

Taking the derivative gives the density:

$$
g(x) = \frac{d}{dx} G_1(x) = 2x
$$


So $g(x) = 2x$ is a better probability density function on $[0, 1]$, and we could sample from it to concentrate more samples where $e^x$ is large, leading to a lower-variance and more efficient importance sampling estimator for the integral.

